In [1]:
from face2voice.models.SpeakerEncoder import SpeakerEncoder
from openvoice.api import ToneColorConverter
import torch
import numpy as np
from face2voice.models.Face2Voice import Face2VoiceModel
from face2voice.models.FaceEncoder import FaceEncoder
from PIL import Image
from torchvision import transforms
from face2voice.losses.compound_loss import CompoundLoss
from face2voice.models.TTSModel import TTSModel

c:\Users\user\Desktop\projects\face2voice\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [12]:
tts_model = TTSModel(language="ru")
tts_model.synthesize(text="Радуга, атмосферное, оптическое и метеорологическое явление, наблюдаемое при освещении ярким источником света множества водяных капель.",
                     output_path="outputs/pipertts_ru_test.wav")

Synthesizing base speech
Base audio generated: outputs/pipertts_ru_test.wav


'outputs/pipertts_ru_test.wav'

In [3]:
speaker_encoder = SpeakerEncoder(config_path=r'face2voice\checkpoints\tone_conv\config.json', ckpt_path=r'face2voice\checkpoints\tone_conv\checkpoint.pth')

c:\Users\user\Desktop\projects\face2voice\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded checkpoint 'face2voice\checkpoints\tone_conv\checkpoint.pth'
missing/unexpected keys: [] []


In [4]:
face_encoder = FaceEncoder()
state_dict = torch.load(r"face2voice\checkpoints\face_encoder\facenet_checkpoint.pth")
face_encoder.load_state_dict(state_dict=state_dict)

<All keys matched successfully>

In [ ]:
face2voice = Face2VoiceModel(face_encoder=face_encoder, speaker_encoder=speaker_encoder)
st_dict = torch.load(r"face2voice\checkpoints\f2v\face2voice_ckpt.pth", weights_only=False)
face2voice.load_state_dict(st_dict["model_state_dict"])
face2voice.eval()

In [6]:
face_transform = transforms.Compose([
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

])

In [13]:
import os

for pic in os.listdir(r"C:\Users\user\Desktop\projects\face2voice\resources"):
    if ".jpg" in pic:
        speaker = pic.replace(".jpg", "")
        img = Image.open(f"resources\{pic}").convert("RGB")
        img = face_transform(img).unsqueeze(0)
        tgt_emb = face2voice(img)
        src_emb = face2voice.speaker_encoder.encode_single(audio="outputs\pipertts_ru_test.wav", input="audio", return_numpy=False)
        tgt_emb = tgt_emb.detach().clone().requires_grad_(True).reshape(1, -1, 1)
        src_emb = src_emb.detach().clone().requires_grad_(True).transpose(1, 2).squeeze(0).reshape(1, -1, 1)
        face2voice.speaker_encoder.tone_color_converter.convert(audio_src_path="outputs\pipertts_ru_test.wav", src_se=src_emb, tgt_se=tgt_emb, output_path=f"outputs/{speaker}_ru_clone.wav")

OpenVoice version: v1
[(0.0, 8.904875)]
after vad: dur = 8.903990929705216
OpenVoice version: v1
[(0.0, 8.904875)]
after vad: dur = 8.903990929705216
OpenVoice version: v1
[(0.0, 8.904875)]
after vad: dur = 8.903990929705216
OpenVoice version: v1
[(0.0, 8.904875)]
after vad: dur = 8.903990929705216
OpenVoice version: v1
[(0.0, 8.904875)]
after vad: dur = 8.903990929705216
